In [ ]:
# Welcome to your new notebook



In [7]:
df_source = spark.read.format('delta').load('Files/dbo/cleaned_customers')
df_source.show()

StatementMeta(, 98d4666a-d6b1-4ebb-b208-5aadf73ef866, 9, Finished, Available, Finished)

+-----------+-----------+---------+---------------+-------------+-----+------+
|customer_id| first_name|last_name|        address|         city|state|   zip|
+-----------+-----------+---------+---------------+-------------+-----+------+
|          1|       John|      Doe|     123 Elm St|      Toronto|   ON|M4B1B3|
|          2|       Jane|    Smith|  456 Maple Ave|       Ottawa|   ON|K1A0B1|
|          3|    Michael|  Johnson|     789 Oak Dr|     Montreal|   QC|H1A1A1|
|          4|      Emily|    Davis|    101 Pine Rd|      Calgary|   AB|T2A0A1|
|          5|      David|   Wilson| 202 Birch Blvd|    Vancouver|   BC|V5K0A1|
|          6|       Emma|    Clark|   505 Cedar St|      Halifax|   NS|B3H0A1|
|          7|      James| Martinez|  606 Spruce Ln|     Winnipeg|   MB|R3C0A1|
|          8|     Olivia|   Garcia|     707 Fir St|     Edmonton|   AB|T5A0A1|
|          9|    William|    Lopez| 808 Redwood Dr|     Victoria|   BC|V8W0A1|
|         10|        Ava| Anderson|909 Cypress Ave| 

In [10]:
spark.sql("drop table if exists scd_customers" )

StatementMeta(, 98d4666a-d6b1-4ebb-b208-5aadf73ef866, 12, Finished, Available, Finished)

DataFrame[]

In [12]:
create_query = """create table scd_customers
	(
		customer_id int,
		first_name varchar(100),
		last_name varchar(100),
		address varchar(100),
		city varchar(25),
		state varchar(50),
		zip varchar(50),
		createdby varchar(50),
		updatedby varchar(50),
		created_date timestamp,
		updated_date timestamp,
        hash_key BIGINT
	)
	using delta
	location 'Tables/dbo/scd/scd_customers'"""
spark.sql(create_query)

StatementMeta(, 98d4666a-d6b1-4ebb-b208-5aadf73ef866, 14, Finished, Available, Finished)

DataFrame[]

In [14]:
from delta.tables import DeltaTable
target_path = "Tables/dbo/scd/scd_customers"
delta_target = DeltaTable.forPath(spark, target_path)
from pyspark.sql.functions import *
df_src1= df_source.withColumn("hash_key",crc32(concat(*df_source.columns)))
display(df_src1)
df_src1=df_src1.alias("src").join(delta_target.toDF().alias("tgt"),((col("src.customer_id")==col
("tgt.customer_id"))&(col("src.hash_key")==col("tgt.hash_key"))),"anti").select(col("src.*"))
df_src1.show()
from pyspark.sql.functions import col
delta_target.alias("tgt").merge(df_src1.alias("src"),"tgt.customer_id = src.customer_id")\
.whenMatchedUpdate(set={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.updated_date":current_timestamp(),"tgt.updatedby":lit("databricks_Updated")
})\
.whenNotMatchedInsert(values={"tgt.customer_id":"src.customer_id","tgt.first_name":"src.first_name","tgt.last_name":"src.last_name","tgt.address":"src.address","tgt.city":"src.city","tgt.state":"src.state","tgt.zip":"src.zip","tgt.hash_key":"src.hash_key","tgt.created_date":current_timestamp(),"tgt.createdby":lit("databricks"),"tgt.updated_date":current_timestamp(),"tgt.updatedby":lit("databricks")}).execute()
display(spark.read.format("delta").option("header","true").load(target_path))

StatementMeta(, 98d4666a-d6b1-4ebb-b208-5aadf73ef866, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1c2e058a-d6e4-47ed-910f-09b4a28d2971)

+-----------+-----------+---------+---------------+-------------+-----+------+----------+
|customer_id| first_name|last_name|        address|         city|state|   zip|  hash_key|
+-----------+-----------+---------+---------------+-------------+-----+------+----------+
|          1|       John|      Doe|     123 Elm St|      Toronto|   ON|M4B1B3|2373314683|
|          2|       Jane|    Smith|  456 Maple Ave|       Ottawa|   ON|K1A0B1|3911254336|
|          3|    Michael|  Johnson|     789 Oak Dr|     Montreal|   QC|H1A1A1|4114944968|
|          4|      Emily|    Davis|    101 Pine Rd|      Calgary|   AB|T2A0A1| 165372536|
|          5|      David|   Wilson| 202 Birch Blvd|    Vancouver|   BC|V5K0A1|3284019540|
|          6|       Emma|    Clark|   505 Cedar St|      Halifax|   NS|B3H0A1|4268191859|
|          7|      James| Martinez|  606 Spruce Ln|     Winnipeg|   MB|R3C0A1|4278935663|
|          8|     Olivia|   Garcia|     707 Fir St|     Edmonton|   AB|T5A0A1| 443479193|
|         

SynapseWidget(Synapse.DataFrame, ab8ef71a-3383-487f-91dc-dfbc2ee7e2b5)

In [15]:
df = spark.sql("SELECT * FROM dev_lakehouse.scd_customers LIMIT 1000")
display(df)

StatementMeta(, 98d4666a-d6b1-4ebb-b208-5aadf73ef866, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dc01844c-9673-4777-991d-ec353a20c05d)